<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/RS_lecture_5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096342 sha256=189083496a128746e4a719520839f9bd7dff96986b69e84ab0b814a23f521f72
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import pandas as pd
import numpy as np

In [ ]:
!wget https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
!unzip master.zip

--2023-07-08 16:46:46--  https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master [following]
--2023-07-08 16:46:46--  https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   1.06M  4.27MB/s    in 0.2s    

2023-07-08 16:46:47 (4.27 MB/s) - ‘master.zip’ saved [1111929]

Archive:  master.zip
dfe2a910caf170a1f0fd2174867169ce737c9dc7
   creating: netology-recsys-master/
   creating: netology-rec

In [ ]:
links = pd.read_csv('netology-recsys-master/lecture-1/links.csv')
movies = pd.read_csv('netology-recsys-master/lecture-1/movies.csv')
ratings = pd.read_csv('netology-recsys-master/lecture-1/ratings.csv')
tags = pd.read_csv('netology-recsys-master/lecture-1/tags.csv')

In [ ]:
# Соединяем таблицы movied и ratings
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [ ]:
# Для того чтоб ыиспользовать surprise нужно чтобы в датасете были столбцы iid, uid, rating
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
# загружаем датафрейм в Dataset с помощью Reader
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [ ]:
# Это алгоритм коллаборативной фильтрации с использованием скрытых факторов
%%time
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

CPU times: user 913 ms, sys: 1.67 ms, total: 914 ms
Wall time: 973 ms


In [ ]:
test_pred = algo.test(testset)

In [ ]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8682


0.8681990336677368

In [ ]:
algo.predict(uid=2.0, iid='Mortal Kombat (1995)').est

2.5895436677191443

In [ ]:
current_user_id = 2.0
# Получим список названий фильмов, которым пользователь 2.0 ставил оценкни
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []
# А теперь получим 2 списка - фильмы, которые он еще нге смотрел, и предполагаемыек оценки для этих фильмов
for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue

    scores.append(algo.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

In [ ]:
sorted(scores)[-10:]  # Это предполагаемые оценки для непросмотренных фильмов, отсортированные в порядке возрастания

[4.290645872171909,
 4.2923942638822075,
 4.312343314655032,
 4.3179390424606865,
 4.328781455972176,
 4.333000674473621,
 4.356912155716694,
 4.392378371522267,
 4.399658887513509,
 4.4029537525049145]

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|')) # Распарсить жанры

In [ ]:
%%time
movie_genres = [change_string(g) for g in movies.genres.values] # Это парсим столбец жанров

CPU times: user 11.2 ms, sys: 2 µs, total: 11.2 ms
Wall time: 13.6 ms


In [ ]:
%%time
movies_with_ratings['genres'] = movies_with_ratings['genres'].apply(change_string) # так выходит дольше

CPU times: user 65.2 ms, sys: 1.94 ms, total: 67.1 ms
Wall time: 70.6 ms


In [ ]:
movie_genres[0]

'Adventure Animation Children Comedy Fantasy'

In [ ]:
# Строим эмбеддинги по жанрам и ищем ближайших соседей
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(movie_genres)
neigh = NearestNeighbors(n_neighbors=20, n_jobs=-1, metric='euclidean')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=20)

In [ ]:
test = change_string("Adventure|Comedy|Fantasy|Crime")
X_tfidf2 = vectorizer.transform([test])
res = neigh.kneighbors(X_tfidf2, return_distance=True)

In [ ]:
res # Получаем индексы по ближайшим соседям

(array([[0.42079615, 0.53300564, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.54288608, 0.54288608, 0.54288608,
         0.54288608, 0.54288608, 0.6188388 , 0.62682864, 0.62682864]]),
 array([[6774, 9096, 2302, 5627, 6723, 8361, 5737, 2608, 7865, 5636, 3302,
         7496, 3582,  863, 3376, 9717, 3576, 2206, 3010, 6133]]))

In [ ]:
movies.iloc[res[1][0]] # и используем их для просмотра записей

,movieId,title,genres
6774,60074,Hancock (2008),Action|Adventure|Comedy|Crime|Fantasy
9096,143559,L.A. Slasher (2015),Comedy|Crime|Fantasy
2302,3052,Dogma (1999),Adventure|Comedy|Fantasy
5627,27251,"10th Kingdom, The (2000)",Adventure|Comedy|Fantasy
6723,58972,Nim's Island (2008),Adventure|Comedy|Fantasy
8361,109042,Knights of Badassdom (2013),Adventure|Comedy|Fantasy
5737,30810,"Life Aquatic with Steve Zissou, The (2004)",Adventure|Comedy|Fantasy
2608,3489,Hook (1991),Adventure|Comedy|Fantasy
7865,94015,Mirror Mirror (2012),Adventure|Comedy|Fantasy
5636,27368,Asterix & Obelix: Mission Cleopatra (Astérix &...,Adventure|Comedy|Fantasy


In [ ]:
movies_with_ratings.sort_values('timestamp', inplace=True) # сортируем рейтинги по времени  отзывов

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
%%time
title_genres = {} # получаем словарь название - жанры

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

<timed exec>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

CPU times: user 627 ms, sys: 4.93 ms, total: 632 ms
Wall time: 644 ms


In [ ]:
title_genres['Toy Story (1995)']

'Adventure|Animation|Children|Comedy|Fantasy'

In [ ]:
%%time
title_genres = movies_with_ratings[['title', 'genres']].drop_duplicates().set_index('title')['genres'].to_dict() # Так в 15 раз быстрее!

CPU times: user 43.1 ms, sys: 0 ns, total: 43.1 ms
Wall time: 43.1 ms


In [ ]:
current_user_id = 5
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

In [ ]:
LAST_N_MOVIES = 5

genres = []
for movie in user_movies[-LAST_N_MOVIES:]:
    genres.append(title_genres[movie])

In [ ]:
unique_genres = set()
for genre in genres:
    genres_i = genre.split('|')
    for parsed_genre in genres_i:
        unique_genres.add(parsed_genre)
unique_genres = list(unique_genres)
out_genre = ""
for genre in sorted(unique_genres):
    out_genre += " "
    out_genre += genre
out_genre = out_genre[1:]

In [ ]:
out_genre

'Action Crime Drama Thriller'

In [ ]:
title_genres[user_movies[-1]]

'Action|Thriller'

In [ ]:
def recommend_for_user(user_id):
    current_user_id = user_id
    # Получаем список фильмов, котрым этот пользователь ставил оценки
    user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()
    # ищем последний просмотренный фильм
    last_user_movie = user_movies[-1]

    # получаем жанры для того (последнего) фильма
    movie_genres = title_genres[last_user_movie]
    # парсим жанры
    movie_genres = change_string(movie_genres)
    # получаем предсказания kNN ( ближайшие соседи - фильмы у которых близок вектор по оценкам и жанрам)
    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict)

    res = neigh.kneighbors(X_tfidf2, return_distance=True)

    # записываем результат в переменную movies_to_score
    movies_to_score = movies.iloc[res[1][0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue
        # итерируемся по результату kNN и получаем оценки рейтинга с помощью svd
        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)


    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(titles[i], scores[i])

In [ ]:
movies_with_ratings[movies_with_ratings.userId == 2.0].sort_values('rating')

,movieId,title,genres,userId,rating,timestamp
97478,114060,The Drop (2014),Crime|Drama|Thriller,2.0,2.0,1.445715e+09
93998,91658,"Girl with the Dragon Tattoo, The (2011)",Drama|Thriller,2.0,2.5,1.445715e+09
8652,318,"Shawshank Redemption, The (1994)",Crime|Drama,2.0,3.0,1.445715e+09
96746,109487,Interstellar (2014),Sci-Fi|IMAX,2.0,3.0,1.445715e+09
91063,77455,Exit Through the Gift Shop (2010),Comedy|Documentary,2.0,3.0,1.445715e+09
90135,71535,Zombieland (2009),Action|Comedy|Horror,2.0,3.0,1.445715e+09
93833,91529,"Dark Knight Rises, The (2012)",Action|Adventure|Crime|IMAX,2.0,3.5,1.445715e+09
95272,99114,Django Unchained (2012),Action|Drama|Western,2.0,3.5,1.445715e+09
97675,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2.0,3.5,1.445715e+09
76960,8798,Collateral (2004),Action|Crime|Drama|Thriller,2.0,3.5,1.445715e+09


In [ ]:
recommend_for_user(2.0)

Taxi Driver (1976) 4.002098345418905
Simple Plan, A (1998) 3.967268045593234
Few Good Men, A (1992) 3.9523380093777236
Limey, The (1999) 3.8968704173876856
Shadow of a Doubt (1943) 3.8170214948180767
Killing Zoe (1994) 3.7957155906921694
Cape Fear (1962) 3.7836821663878273
Rope (1948) 3.757474011584865
Jackie Brown (1997) 3.705843602548442
Lodger: A Story of the London Fog, The (1927) 3.6122183327072506
